In [21]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
np.set_printoptions(linewidth=1000)
np.set_printoptions(threshold=5000)
import pandas as pd
from PIL import Image
import os
import glob

from matplotlib import pyplot as plt
%matplotlib widget

import pycolmap

import trimesh
from python_utils import *
from navi import transformations
import blender_plots as bplt

orange1 = hex_to_rgb(0xff9a00)
orange2 = hex_to_rgb(0xff5d00)
blue1 = hex_to_rgb(0x00a2ff)
blue2 = hex_to_rgb(0x0065ff)

ydown2zup = np.array([
    [0, 0, 1, 0],
    [-1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, 0, 1],
])
yup2zup = np.array([
    [1, 0, 0, 0],
    [0, 0, -1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 0],
])

def umeyama(X, Y):
    # https://github.com/clementinboittiaux/umeyama-python
    mu_x = X.mean(axis=1).reshape(-1, 1)
    mu_y = Y.mean(axis=1).reshape(-1, 1)
    var_x = np.square(X - mu_x).sum(axis=0).mean()
    cov_xy = ((Y - mu_y) @ (X - mu_x).T) / X.shape[1]
    U, D, VH = np.linalg.svd(cov_xy)
    S = np.eye(X.shape[0])
    if np.linalg.det(U) * np.linalg.det(VH) < 0:
        S[-1, -1] = -1
    c = np.trace(np.diag(D) @ S) / var_x
    R = U @ S @ VH
    t = mu_y - c * R @ mu_x
    return c, R, t

def get_navi_transform(camera_info):
    T = transformations.quaternion_to_rotation_matrix(camera_info['q']).numpy()
    T[:3, -1] = camera_info['t']
    T = np.linalg.inv(T)
    return T


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Download the Navi dataset as described here https://github.com/google/navi

and run the downsampling script from probe3d https://github.com/mbanani/probe3d/blob/main/data_processing/resize_navi.py

In [22]:
navi_dir = Path('/home/linus/workspace/data/navi_v1/') # update to match navi dataset location
with open(navi_dir / 'custom_splits/single_image_3d/objects-val.txt') as split_file:
    val_objects = split_file.readlines()
    val_objects = [s.rstrip('\n') for s in val_objects]

In [23]:
object_name = val_objects[0]
input_dir = Path(glob.glob(str(navi_dir / object_name / "multiview_*"))[0])

object_name = 'schleich_lion_action_figure'
object_dir = navi_dir / object_name
input_dir = object_dir / 'multiview_08_canon_t4i'

In [24]:
output_path = Path('outputs')
image_dir = input_dir / 'images'
image_paths = sorted([f for f in os.listdir(image_dir) if (f[-4:] == '.jpg') and ('downsampled' in f)])
database_path = output_path / f"{object_name}_database.db"

In [ ]:
database_path.parent.mkdir(exist_ok=True)
if database_path.exists():
    os.remove(database_path)

pycolmap.extract_features(
    database_path,
    image_dir,
    image_list=image_paths,
    sift_options=pycolmap.SiftExtractionOptions(
        estimate_affine_shape=True,
        domain_size_pooling=True,
    ),
    reader_options=pycolmap.ImageReaderOptions()
)
pycolmap.match_exhaustive(
    database_path,
    sift_options=pycolmap.SiftMatchingOptions(
        guided_matching=True,
    )
)

pipeline_options = pycolmap.IncrementalPipelineOptions()
maps = pycolmap.incremental_mapping(database_path, image_dir, output_path, pipeline_options)
reconstruction = maps[0]

In [ ]:
annotations = pd.read_json(input_dir / "annotations.json")
gt_mesh = trimesh.load(object_dir / f'3d_scan/{object_name}.obj')

T_colmap = np.array([
    np.vstack([image.cam_from_world.inverse().matrix(), [0, 0, 0, 1]])
    for image in reconstruction.images.values()
])
T_gt = np.array([
    get_navi_transform(get_df_single(annotations, filename=image.name.lstrip('downsampled_')).camera)
    for image in reconstruction.images.values()
])
T_normalize = np.block([
    np.eye(3),
])
gt_mean = T_gt[:, :3, -1].mean(axis=0)
gt_max = (T_gt[:, :3, -1] - gt_mean).max()
T_normalize = np.block([
    [np.eye(3) / gt_max, -gt_mean[:, None] / gt_max],
    [0, 0, 0, 1],
])

In [ ]:
s_align, R_align, t_align = umeyama(T_colmap[:, :3, -1].T, T_gt[:, :3, -1].T)
T_align = np.block([
    [R_align * s_align, t_align],
    [0, 0, 0, 1],
])

In [ ]:
bplt.Scatter(
    (yup2zup @ T_normalize @ (T_align @ T_colmap))[:, :3, -1],
    color=orange1,
    marker_scale=0.006,
    name='colmap',
)
bplt.Scatter(
    (yup2zup @ T_normalize @ T_gt)[:, :3, -1],
    color=blue1,
    marker_scale=0.006,
    name='gt',
)
bplt.Arrows(
    (yup2zup @ T_normalize @ (T_align @ T_colmap))[:, :3, -1],
    end=(yup2zup @ T_normalize @ T_gt)[:, :3, -1],
    radius=0.002,
    head_length=0.01
)
bplt.blender_utils.new_mesh(
    np.einsum('ij,...j->...i', yup2zup[:3] @ T_normalize, np.hstack([gt_mesh.vertices, np.ones((len(gt_mesh.vertices), 1))])),
    gt_mesh.faces
)

In [ ]:
def to_transform(colmap_pose):
    return np.block([
        [colmap_pose.matrix()],
        [0, 0, 0, 1],
    ])

points = np.array([yup2zup @ T_normalize @ T_align @ np.array([*p.xyz, 1]) for p in reconstruction.points3D.values()])[..., :-1]
colors = np.array([p.color for p in reconstruction.points3D.values()]) / 255

bplt.Scatter(points, color=colors, radius=0.002, marker_type='ico_spheres')

R_gt = np.array([*annotations.camera.apply(lambda c: transformations.quaternion_to_rotation_matrix(c['q'])[:3, :3].numpy())])
t_gt = np.array([*annotations.camera.apply(lambda c: np.array(c['t']))])
mean_gt = t_gt.mean(axis=0)

for i, image in reconstruction.images.items():
    camera = reconstruction.cameras[image.camera_id]
    colmap_pose = yup2zup @ T_normalize @ T_align @ to_transform(image.cam_from_world.inverse())
    bplt.Scatter(
        colmap_pose[:3, -1],
        marker_rotation=colmap_pose[:3, :3],
        marker_type=bplt.marker_utils.get_frustum(
            intrinsics=camera.calibration_matrix(),
            height=camera.height, width=camera.width,
            image_depth=.1,
            color=orange1,
            color_fill=orange2,
            name=f'camera_{i}',
            thickness=0.01,
        ),
        name=f'camera_{i}',
    )

    gt_info = get_df_single(annotations, filename=image.name.lstrip('downsampled_'))
    gt_intrinsics = np.array([
        [gt_info.camera['focal_length'], 0, gt_info.image_size[1] / 2],
        [0, gt_info.camera['focal_length'], gt_info.image_size[0] / 2],
        [0, 0, 1],
    ])
    gt_pose = yup2zup @ T_normalize @ get_navi_transform(gt_info.camera)
    bplt.Scatter(
        gt_pose[:3, -1],
        marker_rotation=gt_pose[:3, :3],
        marker_type=bplt.marker_utils.get_frustum(
            intrinsics=gt_intrinsics,
            height=gt_info.image_size[0], width=gt_info.image_size[1],
            image_depth=.1,
            color=blue1,
            color_fill=blue2,
            name=f'gt_camera_{i}',
            thickness=0.01,
        ),
        name=f'gt_camera_{i}',
    )

missing_images = [
    image_path for image_path in image_paths
    if image_path not in [image.name for image in reconstruction.images.values()]
]
for i, image_path in enumerate(missing_images):
    gt_info = get_df_single(annotations, filename=image_path.lstrip('downsampled_'))
    gt_intrinsics = np.array([
        [gt_info.camera['focal_length'], 0, gt_info.image_size[1] / 2],
        [0, gt_info.camera['focal_length'], gt_info.image_size[0] / 2],
        [0, 0, 1],
    ])
    gt_pose = yup2zup @ T_normalize @ get_navi_transform(gt_info.camera)
    bplt.Scatter(
        gt_pose[:3, -1],
        marker_rotation=gt_pose[:3, :3],
        marker_type=bplt.marker_utils.get_frustum(
            intrinsics=gt_intrinsics,
            height=gt_info.image_size[0], width=gt_info.image_size[1],
            image_depth=.1,
            color=blue1,
            color_fill=blue2,
            name=f'failed_gt_camera_{i}',
            thickness=0.01,
        ),
        name=f'failed_gt_camera_{i}',
    )


In [20]:
import bpy
bpy.ops.wm.save_as_mainfile(filepath=str(Path(os.getcwd()) / 'output.blend'))

Info: Total files 0 | Changed 0 | Failed 0
Info: Saved "output.blend"


{'FINISHED'}